# Décomposition de Benders

Soit le problème de programmation linéaire $\mathcal{P}$ ci-dessous:
\begin{align*}
\min 3x_1+4x_2+12x_3+4y_1+6y_2&\\
\text{Sous les contraintes}\qquad\qquad\qquad&\\
-x_1+\phantom{2}x_2+\phantom{2}x_3+2y_1+3y_2&= 5\\
x1-2x_2+2x_3+3y_1+5y_2&=11\\
x,y \in \mathbb{R}+\qquad\qquad&
\end{align*}

## 1. Résolution du problème complet avec DOcplex
DOcplex (Decision Optimization with cplex) est une bibliothèque python permettant d'utiliser les solvers CPLEX (programmation linéaire, programmation quadratique, programmation linéaire en nombres entiers) et CPO (programmation par contraintes). Elle constitue une couche supérieure aux API python déjà existantes pour chacun de ces deux solveurs. DOcplex permet de manipuler des modèles qui sont ensuite envoyés aux solvers. Pour ce cours, nous utiliserons uniquement le solveur CPLEX avec la partie *mathematical programming* de DOcplex.

Pour plus d'information, consulter le site suivant: https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/mp/index.html
On trouve des exemples intéressants ici: https://github.com/IBMDecisionOptimization/docplex-examples/tree/master/examples/mp/jupyter

Il existe plusieurs façons de modéliser un problème de programmation linéaire à l'aide de DOcplex. Nous nous en tiendrons à quelque chose de très simple, suffisant pour ce que nous voulons illustrer ici.


In [1]:
from docplex.mp.model import Model

# Déclaration du modèle
pc = Model(name='Probleme complet', log_output=True)

# Variables
x1= pc.continuous_var(name='x1')
x2= pc.continuous_var(name='x2')
x3= pc.continuous_var(name='x3')
y1= pc.continuous_var(name='y1')
y2= pc.continuous_var(name='y2')

# Contraintes
pc.add_constraint(-1*x1 +  x2 +  x3 +2*y1 +3*y2 ==  5)
pc.add_constraint(   x1 -2*x2 +2*x3 +3*y1 +5*y2 == 11)

# Fonction objectif
pc.minimize(3*x1 +4*x2 +12*x3 +4*y1 +6*y2)

Il est possible d'afficher les informations liées au modèle:

In [2]:
pc.print_information()

Model: Probleme complet
 - number of variables: 5
   - binary=0, integer=0, continuous=5
 - number of constraints: 2
   - linear=2
 - parameters: defaults
 - objective: minimize
 - problem type is: LP


ou même le modèle complet:

In [3]:
print(pc.export_as_lp_string())

pc.export_as_lp("example.lp")

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Probleme complet

Minimize
 obj: 3 x1 + 4 x2 + 12 x3 + 4 y1 + 6 y2
Subject To
 c1: - x1 + x2 + x3 + 2 y1 + 3 y2 = 5
 c2: x1 - 2 x2 + 2 x3 + 3 y1 + 5 y2 = 11

Bounds
End



'example.lp'

Pour lancer la résolution:

In [4]:
sol = pc.solve()
pc.print_solution()

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            13.200000
objective: 15.000
  x1=1.000
  y2=2.000


Pour récupérer la valeur d'une variable:

In [5]:
sx1=sol.get_value(x1)
print(sx1)

1.0000000000000009


## 2. Décomposition de Benders
### 2.1 Écriture du problème maître et du sous-problème dual
Nous décomposons le problème $\mathcal{P}$ en conservant les variables $y$ dans le problème maître. 

<div class="alert alert-warning">
Problème maître:

\begin{align*}
\min 4y_1+6y_2+z&\\
y \in \mathbb{R}_+^2,z \in \mathbb{R}_+&
\end{align*}

</div>

<div class="alert alert-warning">

Sous-problème primal:
\begin{align*}
\min 3x_1+4x_2+12x_3&\\
\text{Sous les contraintes}\qquad\qquad\qquad&\\
-x_1+\phantom{2}x_2+\phantom{2}x_3&= 5-2\bar{y_1}-3\bar{y_2}\\
x_1-2x_2+2x_3&=11-3\bar{y_1}-5\bar{y_2}\\
x \in \mathbb{R}_+^3\qquad&
\end{align*}
</div>

<div class="alert alert-warning">

Sous-problème dual:
\begin{align*}
\max (5-2\bar{y_1}-3\bar{y_2})u_1 &+ (11-3\bar{y_1}-5\bar{y_2})u_2\\
\text{Sous les contraintes}\qquad\qquad\qquad&\\
-u_1+\phantom{2}u_2&\leq 3 \\
u_1-2u_2&\leq 4 \\
u_1+2u_2&\leq 12 \\
u \in \mathbb{R}^2\quad&
\end{align*}
</div>

Tracer sur papier l'ensemble des solutions admissibles du sous-problème dual. En déduire le type de coupes qui seront renvoyées vers le problème maître.

### 2.2 Démarche itérative
Dans cette partie nous allons définir et résoudre itérativement le problème maître puis le sous-problème dual, en mettant à jour respectivement les contraintes et la fonction objectif.

**Itération n=0**

In [ ]:
from docplex.mp.model import Model

# Déclaration du modèle
master = Model(name='Probleme maître', log_output=True)

# Variables
y1= master.continuous_var(name='y1')
y2= master.continuous_var(name='y2')
z= master.continuous_var(name='z')

# Fonction objectif
master.minimize(4*y1 +6*y2+z)

# Résolution
master_sol = master.solve()
sy1 = master_sol.get_value(y1)
sy2 = master_sol.get_value(y2)

master.print_solution()

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 3 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
objective: 0.000


In [ ]:
# Déclaration du modèle
worker = Model(name='Sous-problème dual', log_output=True)

# Variables duales
u1 = worker.continuous_var(lb=-worker.infinity, name='u1')
u2 = worker.continuous_var(lb=-worker.infinity, name='u2')

# Contraintes
ct1 = worker.add_constraint(-1*u1 +  u2 <=  3)
ct2 = worker.add_constraint(   u1 -2*u2 <=  4)
ct3 = worker.add_constraint(   u1 +2*u2 <= 12)

# fonction objectif
worker.maximize((5-2*sy1-3*sy2)*u1 + (11-3*sy1-5*sy2)*u2)

# Résolution
worker_sol = worker.solve()
su1 = worker_sol.get_value(u1)
su2 = worker_sol.get_value(u2)

worker.print_solution()


CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.499998
objective: 65.000
  u1=2.000
  u2=5.000


**Convergence**  
Comme $w=65$ et $z=0$, on doit intégrer une coupe d'optimalité au problème maître:
$$2*(5-2y_1-3y_2)+5*(11-3y_1-5y_2)<=z$$

**Itération n=1**

In [ ]:
# Ajout de la coupe au problème maître
master.add_constraint((5-2*y1-3*y2)*su1 + (11-3*y1-5*y2)*su2<=z)

# Résolution
master_sol = master.solve()
sy1 = master_sol.get_value(y1)
sy2 = master_sol.get_value(y2)

master.print_solution()

CPXPARAM_Read_DataCheck                          1

Iteration log . . .
Iteration:     1   Dual objective     =            12.580645
objective: 12.581
  y2=2.097


In [ ]:
# Nouvelle fonction objectif du sous-problème 
worker.maximize((5-2*sy1-3*sy2)*u1 + (11-3*sy1-5*sy2)*u2)

# Résolution
worker_sol = worker.solve()
su1 = worker_sol.get_value(u1)
su2 = worker_sol.get_value(u2)

worker.print_solution()

CPXPARAM_Read_DataCheck                          1
Using devex.

Iteration log . . .
Iteration:     1    Objective     =             9.290323
objective: 9.290
  u1=-10.000
  u2=-7.000


**Convergence**  
Comme $w=9.29$ et $z=0$, on doit intégrer une coupe d'optimalité au problème maître:
$$-10*(5-2y_1-3y_2) -7*(11-3y_1-5y_2)<=z$$

**Itération n=2**

In [ ]:
# Ajout de la coupe au problème maître
master.add_constraint((5-2*y1-3*y2)*su1 + (11-3*y1-5*y2)*su2<=z)

# Résolution
master_sol = master.solve()
sy1 = master_sol.get_value(y1)
sy2 = master_sol.get_value(y2)

master.print_solution()

CPXPARAM_Read_DataCheck                          1

Iteration log . . .
Iteration:     1   Dual objective     =            15.000000
objective: 15.000
  y2=2.000
  z=3.000


In [ ]:
# Nouvelle fonction objectif du sous-problème 
worker.maximize((5-2*sy1-3*sy2)*u1 + (11-3*sy1-5*sy2)*u2)

# Résolution
worker_sol = worker.solve()
su1 = worker_sol.get_value(u1)
su2 = worker_sol.get_value(u2)

worker.print_solution()

CPXPARAM_Read_DataCheck                          1
Using devex.
objective: 3.000
  u1=-10.000
  u2=-7.000


**Convergence**  
Comme $w=z=3$, la borne supérieure et la borne inférieure se sont rejointes, on a atteint la solution optimale.

La solution du problème est donc: objectif=15, $y_1=0$, $y_2=2$  
Pour trouver les valeurs de $x_1$, $x_2$ et $x_3$ à partir de celle de $u_1$ et $u_2$, on peut utiliser le théorème des écarts complémentaires. On peut aussi récupérer directement ces valeurs de la résolution du problème dual. En effet les variables primales $x_1$, $x_2$ et $x_3$ sont les variables duales du problème dual. Si on a fait les choses dans l'ordre, $x_1$ est la variable duale associée à la première contrainte du sous-problème dual, $x_2$ à la deuxième et $x_3$ à la troisième. CPLEX donne accès à ces valeurs lorsque le problème dual est résolu (attention il faut au préalable avoir nommé les contraintes lors de leur définition, ici `ct1`, `ct2` et `ct3`).

In [ ]:
print('x1=',ct1.dual_value)
print('x2=',ct2.dual_value)
print('x3=',ct3.dual_value)

x1= 1.0
x2= 0
x3= 0


Nous retrouvons bien la solution du problème : objectif=15, $y_2=2$ et $x_1=1$ 

### 2.2 Modèle intégré
Développer au sein du même code l'algorithme de Benders qui résout alternativement le problème maître et le sous-problème jusqu'à optimalité.

In [ ]:
# Définition du problème maître restreint
master2 = Model(name='Benders maître')
y1 = master2.continuous_var(name='y1')
y2 = master2.continuous_var(name='y2')
z  = master2.continuous_var(name='z')
master2.minimize(4*y1 +6*y2 +z)

# Définition des contraintes du sous-problème dual
worker2 = Model(name='Benders sous-problème')
worker2.parameters.preprocessing.reduce.set(0)

u1 = worker2.continuous_var(lb=-worker2.infinity, name='u1')
u2 = worker2.continuous_var(lb=-worker2.infinity, name='u2')
ct1 = worker2.add_constraint(-1*u1 +  u2 <=  3)
ct2 = worker2.add_constraint(   u1 -2*u2 <=  4)
ct3 = worker2.add_constraint(   u1 +2*u2 <= 12)

# Boucle de l'algo
# -----------------
n=0
epsilon = 1E-6

boucle = True 
optimal = False

while boucle:
    print()
    print("Itération ",n)
    
    # Résolution du problème maître restreint
    master2_sol = master2.solve()
    if master2_sol is None :
        print("Problème non réalisable")
        break
    master2_obj = master2_sol.get_objective_value()
    
    print("Objectif du master:",master2_obj)
    sy1 = master2_sol.get_value(y1)
    sy2 = master2_sol.get_value(y2)
    sz  = master2_sol.get_value(z)
    print("z=",sz)
    
    # Résolution du sous-problème dual
    worker2.maximize((5-2*sy1-3*sy2)*u1 + (11-3*sy1-5*sy2)*u2)
    worker2_sol = worker2.solve() 
    if worker2.solve_details.status == 'infeasible' :
        print("Sous-problème dual non réalisable: problème non borné")
        break
    elif worker2.solve_details.status == 'unbounded' :
        # Ajout d'une coupe de faisabilité
        print("Sous-problème dual non borné: ajout d'une coupe de faisabilité")
        ray = worker2.get_engine().get_cplex().solution.advanced.get_ray()
        master2.add_constraint((5-2*y1-3*y2)*ray[0] + (11-3*y1-5*y2)*ray[1]<=0)
    else :
        # Ajout d'une coupe d'optimalité au problème maître restreint
        print("Sous-problème dual a une solution bornée: ajout d'une coupe d'optimalité")
        worker2_obj = worker2_sol.get_objective_value()
        print("w=",worker2_obj)
        su1 = worker2_sol.get_value(u1)
        su2 = worker2_sol.get_value(u2)
        master2.add_constraint((5-2*y1-3*y2)*su1 + (11-3*y1-5*y2)*su2<=z)
    
    # Sortie: test d'optimalité
    if worker2_obj - sz <= epsilon:
        boucle = False
        optimal = True
    n=n+1
    
if optimal:
    print()
    print('Solution optimale')
    print('y1=',master2_sol.get_value(y1))
    print('y2=',master2_sol.get_value(y2))
    print('x1=',ct1.dual_value)
    print('x2=',ct2.dual_value)
    print('x3=',ct3.dual_value)
    print('objectif=',master2_sol.get_objective_value())
    
print("Fini !")


Itération  0
Objectif du master: 0.0
z= 0
Sous-problème dual a une solution bornée: ajout d'une coupe d'optimalité
w= 65.0

Itération  1
Objectif du master: 12.580645161290322
z= 0
Sous-problème dual a une solution bornée: ajout d'une coupe d'optimalité
w= 9.290322580645162

Itération  2
Objectif du master: 14.999999999999986
z= 3.0
Sous-problème dual a une solution bornée: ajout d'une coupe d'optimalité
w= 3.0

Solution optimale
y1= 0
y2= 2.0
x1= 1.0
x2= 0
x3= 0
objectif= 14.999999999999986
Fini !
